In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, transform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import mnist

Using TensorFlow backend.


In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
print(train_data.shape,
     test_data.shape)

(42000, 785) (28000, 784)


In [5]:
(x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()
x_train1 = np.concatenate((x_test1, x_train1))
y_train1 = np.concatenate((y_test1, y_train1))

x_train1 = x_train1.reshape((x_train1.shape[0], 28, 28, 1))
print(x_train1.shape, y_train1.shape)

11493376/11490434 [==============================] - 167s 15us/step
(70000, 28, 28, 1) (70000,)


In [4]:
x = np.array(train_data.drop(['label'], axis = 1))
y = np.array(train_data['label'])
test_data = np.array(test_data)

x = x.reshape((x.shape[0], 28, 28, 1))
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

#x = np.concatenate((x, x_train1))
#y = np.concatenate((y, y_train1))

x = x/255
test_data = test_data/255
y = to_categorical(y, num_classes = 10)

print(x.shape, y.shape)

(42000, 28, 28, 1) (42000, 10)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(37800, 28, 28, 1) (37800, 10) (4200, 28, 28, 1) (4200, 10)


In [6]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.20))

model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.30))

model.add(Conv2D(filters = 128, kernel_size = (3,3), activation ='relu'))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.30))
model.add(Dense(10, activation = "softmax"))
optimizer = RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          36928     
__________

In [9]:
datagen = ImageDataGenerator(
        rotation_range = 10,
        zoom_range = 0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,)

train_batch = datagen.flow(x_train, y_train, batch_size = 64)
val_batch = datagen.flow(x_test, y_test, batch_size = 64)

In [10]:
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', 
                                            patience = 3, 
                                            verbose = 1, 
                                            factor = 0.5, 
                                            min_lr = 0.00001)

In [11]:
history = model.fit_generator(generator = train_batch,
                              epochs = 10, validation_data = val_batch,
                              verbose = 1,
                             callbacks = [learning_rate_reduction])

Epoch 1/10
591/591 [==============================] - 134s 226ms/step - loss: 0.3806 - acc: 0.8811 - val_loss: 0.0812 - val_acc: 0.9745
Epoch 2/10
591/591 [==============================] - 139s 235ms/step - loss: 0.1082 - acc: 0.9673 - val_loss: 0.0585 - val_acc: 0.9836
Epoch 3/10
591/591 [==============================] - 140s 236ms/step - loss: 0.0840 - acc: 0.9751 - val_loss: 0.0488 - val_acc: 0.9836
Epoch 4/10
591/591 [==============================] - 138s 233ms/step - loss: 0.0685 - acc: 0.9794 - val_loss: 0.0423 - val_acc: 0.9857
Epoch 5/10
591/591 [==============================] - 139s 235ms/step - loss: 0.0655 - acc: 0.9811 - val_loss: 0.0479 - val_acc: 0.9860
Epoch 6/10
591/591 [==============================] - 142s 239ms/step - loss: 0.0600 - acc: 0.9820 - val_loss: 0.0902 - val_acc: 0.9719
Epoch 7/10
591/591 [==============================] - 146s 247ms/step - loss: 0.0565 - acc: 0.9833 - val_loss: 0.0380 - val_acc: 0.9883
Epoch 8/10
591/591 [============================

In [12]:
res = model.predict_classes(test_data, batch_size = 64)
result = pd.Series(res, name = 'Label')
submission = pd.concat([pd.Series(range(1, 28001), name = 'ImageId'), result], axis = 1)
submission.to_csv('Submission.csv', index = False)

In [13]:
res = model.evaluate(x, y, batch_size = 1024)
print(res[1]*100)

42000/42000 [==============================] - 32s 763us/step
99.23809523809524
